Changing NYT weekdays -> weekends

In [5]:
import pandas as pd
import json
from datetime import datetime


In [6]:
sentimentD = pd.read_csv("Data/DailyDem.csv", encoding = "ISO-8859-1")
sentimentR = pd.read_csv("Data/DailyRep.csv", encoding = "ISO-8859-1")

sentimentD.head()

,date,sentiment score
0,2020-10-01,-0.020733
1,2020-10-02,-0.205017
2,2020-10-03,0.059450
3,2020-10-04,0.083333
4,2020-10-05,0.063640


In [7]:
for index, row in sentimentD.iterrows():
    score = row['sentiment score']
    date = datetime.fromisoformat(row['date'])
    if date.weekday() == 0:
        #it is a Monday, create a combined dict of this day and the last two days, and set it to today
        

        #grab monday, saturday, and sundays data
        saturdayScore = sentimentD.loc[index-2]['sentiment score']
        sundayScore = sentimentD.loc[index-1]['sentiment score']
        mondayScore = score
        
        print(date, saturdayScore, sundayScore, score)
        #copy data from saturday and sunday onto monday
        mondayScore = saturdayScore + sundayScore + score
        
        print("new monday score" , mondayScore)
        
        #drop rows of saturday and sunday
        sentimentD = sentimentD.drop(index=[index-1,index-2], axis=0)
        
        sentimentD.loc[index, 'sentiment score'] =  mondayScore

for index, row in sentimentR.iterrows():
    score = row['sentiment score']
    date = datetime.fromisoformat(row['date'])
    if date.weekday() == 0:
        #it is a Monday, create a combined dict of this day and the last two days, and set it to today
        print(date, score)

        #grab monday, saturday, and sundays data
        saturdayScore = eval(str(sentimentR.loc[index-2]['sentiment score']))
        sundayScore = eval(str(sentimentR.loc[index-1]['sentiment score']))
        mondayScore = score
        
        
        #copy data from saturday and sunday onto monday
        mondayScore = saturdayScore + sundayScore
        
        print("new monday score" , mondayScore)
        
        #drop rows of saturday and sunday
        sentimentR = sentimentR.drop(index=[index-1,index-2], axis=0)
        
        sentimentR.loc[index, 'sentiment score'] =  mondayScore 
#nyt = nyt[ datetime.fromisoformat(eval(nyt['date'])).weekday() != 5 ]
#nyt = nyt[ datetime.fromisoformat(eval(nyt['date'])).weekday() != 6 ]


2020-10-05 00:00:00 0.05945 0.08333333333333333 0.06363999999999999
new monday score 0.2064233333333333
2020-10-12 00:00:00 -0.09895000000000002 0.06433333333333333 0.3062
new monday score 0.27158333333333334
2020-10-19 00:00:00 0.0 -0.09588 -0.17214000000000002
new monday score -0.26802000000000004
2020-10-26 00:00:00 -0.1115 0.08630000000000003 0.11016666666666668
new monday score 0.0849666666666667
2020-11-02 00:00:00 -0.0086 0.3687666666666667 0.025760000000000016
new monday score 0.3859266666666667
2020-11-09 00:00:00 0.21325128205128205 0.22221562499999994 0.117035
new monday score 0.552501907051282
2020-11-16 00:00:00 0.08875 0.025733333333333337 -0.023547058823529414
new monday score 0.0909362745098039
2020-11-23 00:00:00 -0.1806 0.102775 0.1370785714285714
new monday score 0.059253571428571405
2020-10-05 00:00:00 0.014015999999999995
new monday score -0.08950234113712377
2020-10-12 00:00:00 0.11090714285714287
new monday score 0.03604210526315789
2020-10-19 00:00:00 -0.1552687

In [8]:
import yfinance as yf
spy = yf.download("SPY")


[*********************100%***********************]  1 of 1 completed


In [15]:
rowsToDrop = []
for index, row in sentimentD.iterrows():
    try:
        spy.loc[str(row['date'])]
    except:
        print("this doesnt exist", row['date'], index)
        rowsToDrop.append(index)
rowsToDrop

this doesnt exist 2020-11-26 56
this doesnt exist 2020-11-28 58
this doesnt exist 2020-11-29 59


[56, 58, 59]

In [19]:
sentimentD = sentimentD.drop(rowsToDrop)
sentimentR = sentimentR.drop(rowsToDrop)

In [20]:
sentimentD.to_csv("./data/DailyDemFiltered.csv")
sentimentR.to_csv("./data/DailyRepFiltered.csv")